In [4]:
import numpy as np
import pandas as pd
import pandas_profiling

from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

In [5]:
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [17]:
def _regression(x, y, result:list):
    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)
    
    knn = KNeighborsRegressor()
    knn.fit(x_train, y_train)
    res = knn.predict(x_test)
    result.append({'method': 'KNN Regressor', 'score': mean_absolute_error(res, y_test)})
    
    lin = LinearRegression()
    lin.fit(x_train, y_train)
    res = lin.predict(x_test)
    result.append({'method': 'Linear Regressor', 'score': mean_absolute_error(res, y_test)})
    
    dt = DecisionTreeRegressor()
    dt.fit(x_train, y_train)
    res = dt.predict(x_test)
    result.append({'method': 'DT Regressor', 'score': mean_absolute_error(res, y_test)})
    
    rf = RandomForestRegressor()
    rf.fit(x_train, y_train)
    res = rf.predict(x_test)
    result.append({'method': 'RF Regressor', 'score': mean_absolute_error(res, y_test)})


In [9]:
x = pd.read_csv('train_update.csv').drop(['price', 'index'], axis=1)
y = pd.read_csv('train_update.csv')['price']
test = pd.read_csv('test_update.csv').drop(['index'], axis=1)

In [13]:
sc = StandardScaler()
x = sc.fit_transform(x, y)
test = sc.transform(test)

In [69]:
print(x.shape)
y = y.drop(x[(x.restecg == 1) | (x.ca == '?') | (x.thal == '?')].index)
x = x.drop(x[(x.restecg == 1) | (x.ca == '?') | (x.thal == '?')].index)
print(x.shape)


(242, 14)
(237, 14)


In [18]:
result = []
_regression(x, y, result)
print(pd.DataFrame(result))

             method     score
0     KNN Regressor  3.287654
1  Linear Regressor  3.316334
2      DT Regressor  3.508642
3      RF Regressor  2.428086


In [21]:
submit = pd.read_csv('sample_submit.csv')
knn = RandomForestRegressor()
knn.fit(x, y)
res = pd.DataFrame(knn.predict(test), columns=['price'])
submit.price = res
submit.to_csv('submit.csv', index=None)